# SBV2の重みファイルをONNXに変換するGoogle Colabスクリプト

SBV2リポジトリのdevブランチを利用します。
ONNXに変換するpythonスクリプトが、mainブランチにはないためです。

今後、mainブランチに統合されるような形になれば、変更が必要です。

# Google Driveのマウント
認証が入るので、最初のセルに置くのをオススメします。

In [ ]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

# 必要パッケージのインストール

In [ ]:
# このセルを実行して環境構築してください。
# エラーダイアログ「WARNING: The following packages were previously imported in this runtime: [pydevd_plugins]」が出るが「キャンセル」を選択して続行してください。

import os

os.environ["PATH"] += ":/root/.cargo/bin"

!curl -LsSf https://astral.sh/uv/install.sh | sh
!git clone https://github.com/litagin02/Style-Bert-VITS2.git
!git checkout dev
%cd Style-Bert-VITS2/
!uv pip install --system -r requirements.txt
!python initialize.py --skip_default_models

# 各種パスやモデル名の指定

In [ ]:
model_name = "amitaro"
safetensor = "amitaro_e18_s6000.safetensors"

# Google Driveでのパスを特定する
import glob
import os
drive_path = os.path.dirname(glob.glob('/content/drive/MyDrive/colab_SBV2train_sample/SBV2-ONNXconvert.ipynb', recursive=True)[0])
print(drive_path)

# 学習に必要なファイルや途中経過が保存されるディレクトリ
dataset_root = f"{drive_path}/Data"

# 学習結果（音声合成に必要なファイルたち）が保存されるディレクトリ
assets_root = f"{drive_path}/model_assets"

import yaml
with open("configs/paths.yml", "w", encoding="utf-8") as f:
    yaml.dump({"dataset_root": dataset_root, "assets_root": assets_root}, f)

# ONNX形式に重みファイルを変換する
Aivis Speechで利用する場合は、AIVM / AIVMX ファイルを用意する必要がありますが、そのためには学習結果として出力される「Safetensors」形式のモデルだけではなく、「ONNX」形式のモデルも必要になるため、ここで変換をしておきます。

（そのために、SBV2のdevブランチを利用しています。）

In [ ]:
model_safetensor = f"{assets_root}/{model_name}/{safetensor}"

!python convert_onnx.py --model {model_safetensor}

# 実行後Colabノードブックのランタイムを削除する

特に、有料版のColabでは、接続時間に応じてコンピューティングユニットが消費されてしまうので、節約のため、学習完了したら即座にColabのランタイムを削除する必要がある。

下記セルにおいて、`FLAG = True`として、実行すると、ランタイムが接続解除される

In [ ]:
# ノートブックの解放
FLAG = False

from google.colab import runtime

if FLAG:
    runtime.unassign()